# Leveraging Linguistics

## Getting Started

In [ ]:
!conda install -y spacy 
# !pip install spacy

In [ ]:
!python -m spacy download en_core_web_lg

If there is an error above, you can use the smaller model as well. 

Try:
- Windows Shell:```python -m spacy download en``` as **Administrator**
- Linux Terminal:```sudo python -m spacy download en ```

In [4]:
import spacy
from spacy import displacy # for visualization
nlp = spacy.load('en_core_web_lg')

In [4]:
spacy.__version__

'2.0.11'

**Introducing textacy**:

In [5]:
!conda install -c conda-forge textacy 
# !pip install textacy

In [6]:
import textacy

## Redacting Names with Named Entity Recognition

In [7]:
text = "Madam Pomfrey, the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. Ginny Weasley, who had been looking pale, was bullied into taking some by Percy."

In [8]:
# Parse the text with spaCy. This runs the entire NLP pipeline.
doc = nlp(text)

'doc' now contains a parsed version of text. We can use it to do anything we want!. 
For example, this will print out all the named entities that were detected:

In [9]:
for entity in doc.ents:
    print(f"{entity.text} ({entity.label_})")

Pomfrey (PERSON)
Pepperup (ORG)
several hours (TIME)
Ginny Weasley (PERSON)
Percy (PERSON)


In [10]:
doc.ents

(Pomfrey, Pepperup, several hours, Ginny Weasley, Percy)

In [11]:
entity.label, entity.label_

(378, 'PERSON')

In spaCy, all human readable labels etc can also be explained using the simple spacy.explain(label) syntax:

In [12]:
spacy.explain('GPE')

'Countries, cities, states'

In [13]:
def redact_names(text):
    doc = nlp(text)
    redacted_sentence = []
    for token in doc:
        if token.ent_type_ == "PERSON":
            redacted_sentence.append("[REDACTED]")
        else:
            redacted_sentence.append(token.string)
    return "".join(redacted_sentence)

In [14]:
redact_names(text)

'Madam [REDACTED], the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. [REDACTED][REDACTED], who had been looking pale, was bullied into taking some by [REDACTED].'

In [15]:
def redact_names(text):
    doc = nlp(text)
    redacted_sentence = []
    for ent in doc.ents:
        ent.merge()
    for token in doc:
        if token.ent_type_ == "PERSON":
            redacted_sentence.append("[REDACTED]")
        else:
            redacted_sentence.append(token.string)
    return "".join(redacted_sentence)

In [16]:
redact_names(text)

'Madam [REDACTED], the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. [REDACTED], who had been looking pale, was bullied into taking some by [REDACTED].'

### Entity Types 

Let's look at some examples of above in real world sentences, we will also use the `spacy.explain()` on all entities to build a quick mental model of how these things work. 

In [17]:
def explain_text_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        print(f'{ent}, Label: {ent.label_}, {spacy.explain(ent.label_)}')

In [18]:
explain_text_entities('Tesla has gained 20% market share in the months since')

Tesla, Label: ORG, Companies, agencies, institutions, etc.
20%, Label: PERCENT, Percentage, including "%"
the months, Label: DATE, Absolute or relative dates or periods


In [19]:
explain_text_entities('Taj Mahal built by Mughal Emperor Shah Jahan stands tall on the banks of Yamuna in modern day Agra, India')

Taj Mahal, Label: PERSON, People, including fictional
Mughal, Label: NORP, Nationalities or religious or political groups
Shah Jahan, Label: PERSON, People, including fictional
Yamuna, Label: LOC, Non-GPE locations, mountain ranges, bodies of water
Agra, Label: GPE, Countries, cities, states
India, Label: GPE, Countries, cities, states


In [20]:
explain_text_entities('Ashoka was a great Indian king')

Ashoka, Label: PERSON, People, including fictional
Indian, Label: NORP, Nationalities or religious or political groups


In [21]:
explain_text_entities('The Ashoka University sponsors the Young India Fellowship')

Ashoka University, Label: ORG, Companies, agencies, institutions, etc.
the Young India Fellowship, Label: ORG, Companies, agencies, institutions, etc.


## Automatic Question Generation

### Part-of-Speech Tagging

In [22]:
example_text = 'Bansoori is an Indian classical instrument. Tom plays Bansoori and Guitar.'

In [23]:
doc = nlp(example_text)

We need noun chunks. Noun chunks are _noun phrases_ - not a single word, but a short phrase which describes the noun. For example, "the blue skies" or "the world’s largest conglomerate". 

To get the noun chunks in a document, simply iterate over `doc.noun_chunks`: 

In [24]:
for idx, sentence in enumerate(doc.sents):
    for noun in sentence.noun_chunks:
        print(f'sentence{idx+1}', noun)

sentence1 Bansoori
sentence1 an Indian classical instrument
sentence2 Tom
sentence2 Bansoori
sentence2 Guitar


Our example text has two sentences, we can pull out noun phrase chunks from each sentence. We pull out noun phrases instead of single words. This means, we are able to pull out 'an Indian classical instrument' as one noun. This is quite useful as we will see in a moment.  

Next, let's take a quick look at all parts-of-speech tags in our example text. We will use the verbs and adjectives to write some simple question generating logic. 

In [25]:
for token in doc:
    print(token, token.pos_, token.tag_)

Bansoori PROPN NNP
is VERB VBZ
an DET DT
Indian ADJ JJ
classical ADJ JJ
instrument NOUN NN
. PUNCT .
Tom PROPN NNP
plays VERB VBZ
Bansoori PROPN NNP
and CCONJ CC
Guitar PROPN NNP
. PUNCT .


Notice that here 'instrument' is tagged as a NOUN while 'Indian' and 'classical' are tagged as adjectives. This makes sense. Addititionally, Bansoori and Guitar are tagged as PROPN or Proper Nouns. 

**Nouns vs Proper Noun** 
Nouns name people, places, and things. Common nouns name general items like waiter, jeans, country. Proper nouns name specific things like Roger, Levi's, India

### Creating a Ruleset

Quite often when using linguistics, you will be writing custom rules. Here is one data structure suggestion to help you store these rules: list of dictionaries. Each dictionary in turn can have elements ranging from simple string lists to lists to strings. Avoid nesting a list of dictionaries inside a dictionary:

In [26]:
ruleset = [
    {
        'id': 1, 
        'req_tags': ['NNP', 'VBZ', 'NN'],
    }, 
    {
        'id': 2, 
        'req_tags': ['NNP', 'VBZ'],
    }
    ]

Here, I have written two rules. Each rule is simply a collection of part-of-speech tags stored under the 'req_tags' key. Each rule comprises of all the tags that I will look for in a particular sentence. 

Depending on 'id', I will use a hard coded question template to generate my questions. In practice, you can and should move the question template to your ruleset.  

In [27]:
print(ruleset)

[{'id': 1, 'req_tags': ['NNP', 'VBZ', 'NN']}, {'id': 2, 'req_tags': ['NNP', 'VBZ']}]


Next, I need a function to pull out all tokens which match a particular tag. We do this by simply iterating over the entire list of and matching each token against the target tag. 

In [28]:
def get_pos_tag(doc, tag):
    return [tok for tok in doc if tok.tag_ == tag]

In [29]:
def sent_to_ques(sent:str)->str:
    """
    Return a question string corresponding to a sentence string using a set of pre-written rules
    """
    doc = nlp(sent)
    pos_tags = [token.tag_ for token in doc]
    for idx, rule in enumerate(ruleset):
        if rule['id'] == 1:
            if all(key in pos_tags for key in rule['req_tags']): 
                print(f"Rule id {rule['id']} matched for sentence: {sent}")
                NNP = get_pos_tag(doc, "NNP")
                NNP = str(NNP[0])
                VBZ = get_pos_tag(doc, "VBZ")
                VBZ = str(VBZ[0])
                ques = f'What {VBZ} {NNP}?'
                return(ques)
        if rule['id'] == 2:
            if all(key in pos_tags for key in rule['req_tags']): #'NNP', 'VBZ' in sentence.
                print(f"Rule id {rule['id']} matched for sentence: {sent}")
                NNP = get_pos_tag(doc, "NNP")
                NNP = str(NNP[0])
                VBZ = get_pos_tag(doc, "VBZ")
                VBZ = str(VBZ[0].lemma_)
                ques = f'What does {NNP} {VBZ}?'
                return(ques)

In [30]:
for sent in doc.sents:
    print(f"The generated question is: {sent_to_ques(str(sent))}")

Rule id 1 matched for sentence: Bansoori is an Indian classical instrument.
The generated question is: What is Bansoori?
Rule id 2 matched for sentence: Tom plays Bansoori and Guitar.
The generated question is: What does Tom play?


# Question Generation using Dependency Parsing

In [31]:
for token in doc:
    print(token, token.dep_)

Bansoori nsubj
is ROOT
an det
Indian amod
classical amod
instrument attr
. punct
Tom nsubj
plays ROOT
Bansoori dobj
and cc
Guitar conj
. punct


In [32]:
for token in doc:
    print(token, token.dep_, spacy.explain(token.dep_))

Bansoori nsubj nominal subject
is ROOT None
an det determiner
Indian amod adjectival modifier
classical amod adjectival modifier
instrument attr attribute
. punct punctuation
Tom nsubj nominal subject
plays ROOT None
Bansoori dobj direct object
and cc coordinating conjunction
Guitar conj conjunct
. punct punctuation


## Visualizing the Relationship

spaCy has an inbuilt tool called displacy for displaying simple, but clean and powerful visualizations. It offers two primary modes: Named Entity Recognition and Dependency Parsing. Here we will use the 'dep' or dependency mode. 

In [33]:
displacy.render(doc, style='dep', jupyter=True)

In [82]:
tricky_doc = nlp('This is ship-shipping ship, shipping shipping ships')

In [96]:
displacy.render(tricky_doc, style='dep', jupyter=True)

In [85]:
from textacy.spacier import utils as spacy_utils

In [49]:
??spacy_utils.get_main_verbs_of_sent

In [50]:
# Signature: spacy_utils.get_main_verbs_of_sent(sent)
# Source:   
# def get_main_verbs_of_sent(sent):
#     """Return the main (non-auxiliary) verbs in a sentence."""
#     return [tok for tok in sent
#             if tok.pos == VERB and tok.dep_ not in constants.AUX_DEPS]
# File:      d:\miniconda3\envs\nlp\lib\site-packages\textacy\spacier\utils.py
# Type:      function

In [86]:
toy_sentence = 'Shivangi is an engineer'
doc = nlp(toy_sentence)

What are the entities in this sentence? 

In [89]:
displacy.render(doc, style='ent', jupyter=True)

Let's find out the main verb in this sentence: 

In [87]:
verbs = spacy_utils.get_main_verbs_of_sent(doc)
print(verbs)

[is]


And what are nominal subjects of this verb?   

In [88]:
for verb in verbs:
    print(verb, spacy_utils.get_subjects_of_verb(verb))

is [Shivangi]


You will notice that this has a reasonable overlap with the noun phrases which we pulled from our part-of-speech tagging but can be different as well. 

In [90]:
[(token, token.tag_) for token in doc]

[(Shivangi, 'NNP'), (is, 'VBZ'), (an, 'DT'), (engineer, 'NN')]

Tip: As an exercise, extend this approach to at least add Who, Where and When questions as practice. 

# Level Up: Question and Answer
So far, we have been trying to generate questions. But if you were trying to make an automated quiz for students, you would also need to mine the right answer. 

The answer in this case will be simply the objects of verb. What is an object of verb? 

> In the sentence, "Give the book to me," "book" is the direct object of the verb "give," and "me" is the indirect object. - from the Cambridge English Dictionary

Loosely, object is the piece on which our verb acts. This is almost always the answer to our "what". Let's write a question to find the objects of any verb --- or wait, we can pull it from the `textacy.spacier.utils`. 

In [91]:
spacy_utils.get_objects_of_verb(verb)

[engineer]

In [92]:
for verb in verbs:
    print(verb, spacy_utils.get_objects_of_verb(verb))

is [engineer]


In [93]:
displacy.render(doc, style='dep', jupyter=True)

Let's look at the output of our functions for the example text. The first is the sentence itself, then the root verb, than the lemma form of that verb, followed by subjects of the verb and then objects.  

In [95]:
doc = nlp(example_text)
for sentence in doc.sents:
    print(sentence, sentence.root, sentence.root.lemma_, spacy_utils.get_subjects_of_verb(sentence.root), spacy_utils.get_objects_of_verb(sentence.root))

Bansoori is an Indian classical instrument. is be [Bansoori] [instrument]
Tom plays Bansoori and Guitar. plays play [Tom] [Bansoori, Guitar]


Let's arrange the pieces above into a neat function which we can then re-use

In [56]:
def para_to_ques(eg_text):
    doc = nlp(eg_text)
    results = []
    for sentence in doc.sents:
        root = sentence.root
        ask_about = spacy_utils.get_subjects_of_verb(root)
        answers = spacy_utils.get_objects_of_verb(root)
        if len(ask_about) > 0 and len(answers) > 0:
            if root.lemma_ == "be":
                question = f'What {root} {ask_about[0]}?'
            else:
                question = f'What does {ask_about[0]} {root.lemma_}?'
            results.append({'question':question, 'answers':answers})
    return results

In [57]:
para_to_ques(example_text)

[{'question': 'What is Bansoori?', 'answers': [instrument]},
 {'question': 'What does Tom play?', 'answers': [Bansoori, Guitar]}]

This seems right to me. Let's run this on a larger sample of sentences. This sample has varying degrees of complexities and sentence structures. 

In [42]:
large_example_text = """
Puliyogare is a South Indian dish made of rice and tamarind. 
Priya writes poems. Shivangi bakes cakes. Sachin sings in the orchestra.

Osmosis is the movement of a solvent across a semipermeable membrane toward a higher concentration of solute. In biological systems, the solvent is typically water, but osmosis can occur in other liquids, supercritical liquids, and even gases.
When a cell is submerged in water, the water molecules pass through the cell membrane from an area of low solute concentration to high solute concentration. For example, if the cell is submerged in saltwater, water molecules move out of the cell. If a cell is submerged in freshwater, water molecules move into the cell.

Raja-Yoga is divided into eight steps. The first is Yama. Yama is nonviolence, truthfulness, continence, and non-receiving of any gifts.
After Yama, Raja-Yoga has Niyama. cleanliness, contentment, austerity, study, and self - surrender to God.
The steps are Yama and Niyama. 
"""


In [43]:
para_to_ques(large_example_text)

[{'question': 'What is Puliyogare?', 'answers': [dish]},
 {'question': 'What does Priya write?', 'answers': [poems]},
 {'question': 'What does Shivangi bake?', 'answers': [cakes]},
 {'question': 'What is Osmosis?', 'answers': [movement]},
 {'question': 'What is solvent?', 'answers': [water]},
 {'question': 'What is first?', 'answers': [Yama]},
 {'question': 'What is Yama?',
  'answers': [nonviolence, truthfulness, continence, of]},
 {'question': 'What does Yoga have?', 'answers': [Niyama]},
 {'question': 'What are steps?', 'answers': [Yama, Niyama]}]

# Facts Extraction using Semi Structured Sentence Parsing
Introducing textacy,

Boss mode with co reference resolution